In [ ]:
import os       as os
import numpy    as np
import datetime as dt
import sys      as sys

##
## Graphical libs and setup
import matplotlib.pyplot as plt
plt.ioff()
 
#print(plt.style.available)
plt.style.use(['ggplot', 'fast'])
plt.rc('image', cmap='cubehelix') # See https://www.mrao.cam.ac.uk/~dag/CUBEHELIX/
 
import seaborn  as sns
import pandas   as pd
pd.options.display.width = 250
 
from bnzds.utilities import CommonNotebook as cn
from bnzds.utilities import CommonIO       as cio
from bnzds.utilities import CommonGraphs   as cg
from bnzds.utilities import CommonSpark    as cs
 
# Not yet guaranteed to work under phttp://pxlbig03:10020/notebooks/DS_Python_Utilities/bnzds/examples/Jupyter/Lab_Test_BoxGraph.ipynb#ython2
# ML libs should be on python3 anyway
#if sys.version_info[0] >= 3:
#    from bnzds.utilities import CommonML       as cml
#    from bnzds.utilities import CommonTA       as cta
 
## Set OUTPUT paths
mainpath = os.environ.get('DSOUTPUT')
if mainpath is None:
    from os.path import expanduser
    mainpath = expanduser("~") + '/output/'

dpath = mainpath + '/data/'
gpath = mainpath + '/graphs/'
mpath = mainpath + '/model/'
                
## Platform Init
cn.platformInit(libs=[cn, cio], gpath=gpath)
#print = cn.aprint

In [ ]:
pip install duckdb

In [ ]:
import duckdb
con1 = duckdb.connect(':memory:', config={'allow_unsigned_extensions' : 'true'})

# Edge node has 504G
con1.sql("SET memory_limit = '100GB'")  

# See https://stackoverflow.com/questions/71952623/reading-partitioned-parquet-files-in-duckdb
con1.sql("SET temp_directory = '/data/disk1/tmp/duckdbcaches/" + os.environ.get('USER') + "'")

# Use progress bar (if possible)
print(con1.sql("PRAGMA enable_print_progress_bar"))


## Setting up data

In [ ]:
df_fin = pd.read_csv('business-financial-data-march-2024-csv.csv')
df_fin.columns = [str(col).lower().replace(' ','_') for col in df_fin.columns]
df_fin['year'] = df_fin['period'].astype(str).str.slice(0,4).astype(int)
df_fin.rename(columns={'group':'ind_group'}, inplace='True')
df_fin.head()

In [ ]:
df_emp = pd.read_csv('machine-readable-business-employment-data-mar-2024-quarter.csv')
df_emp.columns = [str(col).lower().replace(' ','_') for col in df_emp.columns]
df_emp.rename(columns={'group':'ind_group'}, inplace='True')
df_emp.head()

In [ ]:
duckdb.register('fin',df_fin)
duckdb.register('emp',df_emp)

In [ ]:
print(df_fin.dtypes)

## Question One

In [ ]:
Q1_query = """
with sal_wgs as
(--Get all data where series_title_1 = salaries and wages
select 
    *
from fin 

where series_title_1 = 'Salaries and wages'
      
)

, first_year as
(--All industries where first year for salaries and wages after 2016
select
    series_title_2
    , ind_group
    , min(year) as first_year
from sal_wgs

group by series_title_2, ind_group

having min(year) > 2016
)

select
    series_title_2 as industry
    ,avg_filled_jobs as overall_filled_jobs
from 
(--Average actual filled jobs per industry
    select
        emp.series_title_2
        ,avg(emp.data_value) as avg_filled_jobs
        ,emp.series_title_1
    from emp 
    
    inner join first_year fst on fst.series_title_2 = emp.series_title_2 --Noted that emp dataset only includes NZSIOC Level 2
    --Actual filled jobs only
    where emp.series_title_1 = 'Filled jobs'
          and emp.series_title_3 = 'Actual'

    group by emp.series_title_2
            , emp.series_title_1
) mx

order by avg_filled_jobs desc

limit 1
;
"""

result = duckdb.sql(Q1_query).df()
result

## Question Two

In [ ]:
Q2_query = """
with base as 
(--Get all data for business industry is level 2, and variables are seasonally adjusted operating income
select 
    *
from fin
where series_title_1 = 'Sales (operating income)'
      and ind_group = 'Industry by financial variable (NZSIOC Level 2)'
      and series_title_4 = 'Seasonally adjusted'
)


select 
    period
    ,series_title_2 as industry
    ,data_value as operating_income
from
    (--Rank seasonally adjusted operating income largest to smallest across all periods and industries
    select
        period
        ,series_title_2
        ,data_value
        ,dense_rank() over (order by data_value desc) as operating_inc_rnk
    from base
    ) rnk
where operating_inc_rnk = 2
"""

result = duckdb.sql(Q2_query).df()
result

## Question Three

In [ ]:
Q3_query = """
with 
base as 
(--Get base data for territorial authorities and filled jobs
select
      series_title_2
      ,data_value
      ,period
from emp
where ind_group = 'Territorial authority by employment variable'
      and series_title_1 = 'Filled jobs'
)

,highest_avg as
(--Get the territorial authority with the highest average value for filled jobs 
select
      series_title_2 as territorial_authority
      ,avg_filled_jobs
from      
      (
      select
            series_title_2
            ,avg(data_value) as avg_filled_jobs
      from base
      group by series_title_2
      ) avrg
order by avg_filled_jobs desc
limit 1      
)

, sorted_ta as
(
select
      base.period
      ,ha.territorial_authority
      ,base.data_value as filled_jobs
from highest_avg as ha
inner join base on base.series_title_2 = ha.territorial_authority
)

--Calculate quarterly cumulative number of filled jobs over time
select      
      sta.period
      ,sta.filled_jobs
      ,sum(prd.filled_jobs) as cumulative_filled_jobs
from sorted_ta sta
inner join sorted_ta prd on prd.period <= sta.period
group by sta.period, sta.filled_jobs
order by sta.period

;
"""

result = duckdb.sql(Q3_query).df()
result

## Question Four

We could use libraries like Pandas to check the data for such aberrations, and that they only include values we expect.

For duplicates, we should first check if there are any duplicates, and drop them if so.
/n
e.g. df_fin[df_fin.duplicated() == True] to check for duplicates
     df_fin.drop_duplicates to drop duplicates.
/n
To identify incorrect data types, we would first want to see what data types the datasets contain, and if they are what we expected, using df.dtypes.
Since the datasets are part of a pipeline, we should know what columns and data types we expect to see for each.
Thus, we can first define a dictionary showing the columns and their expected data types.
For example, using the financial data csv, we may define something like the below, and then check the actual data types of our dataset against this dictionary.

In [ ]:
expected_schema = {
    'series_reference': 'object',
    'period': 'object',
     'data_value': 'float64',
     'suppressed': 'object',
     'status': 'object',
     'units': 'object',
     'magnitude': 'int64',
     'subject': 'object',
     'ind_group': 'object',
     'series_title_1': 'object',
     'series_title_2': 'object',
     'series_title_3': 'object',
     'series_title_4': 'object',
     'series_title_5': 'object',
     'year': 'int64'
}

#Then, we would want to check the actual data types of our dataset against this dictionary using:

for col, dtype in expected_schema.items():
    assert df_fin[col].dtype == dtype

When we run this, we will see an AssertionError because period and series_title_5 are not our expected data types of object.
To improve the pipeline, a function could then be written to compare the expected data types against the actual, and if they are different, then cast the actual to the expected.
e.g. The function could be something like:

In [ ]:
def enforce_schema(df_fin, expected_schema):
    for col, expected_dtype in expected_schema.items():
        actual_dtype = df_fin[col].dtype

        # Skip if it's already correct
        if str(actual_dtype) == expected_dtype:
            continue

        print(f"[INFO] Column '{col}' is {actual_dtype}, expected {expected_dtype}. Attempting to cast...")

        try:
            if 'float' in expected_dtype:
                df_fin[col] = pd.to_numeric(df_fin[col], errors='coerce')
            elif 'int' in expected_dtype:
                df_fin[col] = pd.to_numeric(df_fin[col], errors='coerce').astype('Int64')  # nullable int
            elif expected_dtype == 'object':
                df_fin[col] = df_fin[col].astype(str)
            elif 'datetime' in expected_dtype:
                df_fin[col] = pd.to_datetime(df_fin[col], errors='coerce')
            else:
                print(f"[WARN] Unhandled type: {expected_dtype}")
        except Exception as e:
            print(f"[ERROR] Failed to cast column '{col}' to {expected_dtype}: {e}")

    return df_fin


In [ ]:
df_fin_stg = enforce_schema(df_fin,expected_schema)
print(df_fin.dtypes)